In [3]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import warnings
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller

In [ ]:
df_poezdki1 = df_poezdki.groupby(by=['station','line','hour'])[['num_val','date']].agg({'num_val' : 'sum', 'date' : 'max'})
df_poezdki1=df_poezdki1.reset_index()
df_poezdki1['download_station'] = df_poezdki1.groupby(by=['station','hour'])['num_val'].transform('max')
df_poezdki1['mean_bandwidth'] = df_poezdki1.groupby(by=['station','hour'])['num_val'].transform('mean')
df_poezdki1['bandwidth'] = df_poezdki1['download_station']*100/df_poezdki1['mean_bandwidth']

df_poezdki1
from sklearn.cluster import KMeans
model = KMeans(n_clusters=3, random_state=42, n_init=300)
from sklearn.preprocessing import LabelEncoder
le1 = LabelEncoder()
df_poezdki1['station']=le1.fit_transform(df_poezdki1['station'])
le2 = LabelEncoder()
df_poezdki1['line']=le2.fit_transform(df_poezdki1['line'])
le2 = LabelEncoder()
df_poezdki1['date']=le2.fit_transform(df_poezdki1['date'])
cluster= model.fit_predict(df_poezdki1)
df_poezdki1['cluster'] = cluster

df = df_poezdki1.copy()
df['labels'] = model.labels_
df['labels'] = df.labels.astype('category')
pd.set_option('display.max_rows', 80) # для удосбтва анализа делаем
df.groupby('labels', observed=True).agg(['count','mean', 'std']).T


# Загрузка набора данных 

In [4]:
#Разделяем на X и y
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Разделение данных на обучающую и тестовую выборки

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3.1 Обучение модели

# Логистическая регрессия


In [6]:
log_reg = LogisticRegression(max_iter=200)
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)

# Решающее дерево


In [7]:
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
accuracy_tree = accuracy_score(y_test, y_pred_tree)

# Случайный лес

In [8]:
forest_clf = RandomForestClassifier(n_estimators=100)
forest_clf.fit(X_train, y_train)
y_pred_forest = forest_clf.predict(X_test)
accuracy_forest = accuracy_score(y_test, y_pred_forest)


# Результаты

In [9]:
print("Accuracy of Logistic Regression:", accuracy_log_reg)
print("Accuracy of Decision Tree:", accuracy_tree)
print("Accuracy of Random Forest:", accuracy_forest)

Accuracy of Logistic Regression: 1.0
Accuracy of Decision Tree: 1.0
Accuracy of Random Forest: 1.0


# 3.2 Организация непрерывного обучения

**Dag**

In [ ]:

from datetime import datetime, timedelta
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
import pandas as pd
import sqlite3
from sklearn.cluster import KMeans
import joblib

# Функция для загрузки новых данных из базы данных
def load_new_data():
    conn = sqlite3.connect('transport.db')
    query = "SELECT * FROM trips WHERE timestamp > (SELECT MAX(timestamp) FROM model_training_log)"
    new_data = pd.read_sql_query(query, conn)
    conn.close()
    return new_data

# Функция для дообучения модели
def retrain_model():
    # Загружаем новые данные
    new_data = load_new_data()
    
    if new_data.empty:
        print("No new data available for training.")
        return
    
    # Предобработка данных
    scaler = joblib.load('scaler.pkl')
    kmeans = joblib.load('kmeans_model.pkl')
    new_data_scaled = scaler.transform(new_data[['val_num', 'hour', 'metriki', 'input_stair', 'output_stair', 'count_stair',
                                                  'input_door', 'output_door', 'count_door', 'input_escalator', 'output_escalator', 'count_escalator']])
    
    # Дообучение модели
    kmeans.partial_fit(new_data_scaled)
    
    # Сохраняем обновленную модель
    joblib.dump(kmeans, 'kmeans_model.pkl')
    
    # Обновляем лог дообучения
    conn = sqlite3.connect('transport.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO model_training_log (timestamp) VALUES (datetime('now'))")
    conn.commit()
    conn.close()

# Создаем DAG
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'start_date': datetime(2023, 1, 1),
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5)
}

dag = DAG(
    'retrain_model_dag',
    default_args=default_args,
    description='Retrain model every 5 minutes',
    schedule_interval=timedelta(minutes=5),
)

# Операторы для выполнения задач
retrain_task = PythonOperator(
    task_id='retrain_model',
    python_callable=retrain_model,
    dag=dag,
)

retrain_task


# 3.3 Прогнозирование динамики изменения характеристик


In [ ]:
df=df_poezdki.copy()
df=df.sort_values(by='date')
df.set_index('date', inplace=True)

# Убедитесь, что данные упорядочены по дате
df = df.asfreq('D').fillna(0)  # Заполнение пропущенных значений нулями

In [ ]:


# Игнорирование предупреждений, связанных с ARIMA
warnings.filterwarnings("ignore")

# Проверка стационарности временного ряда с помощью теста Дики-Фуллера
result = adfuller(df['num_val'])
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')

# Дифференцирование для получения стационарного временного ряда
df['total_load_diff'] = df['num_val'].diff().dropna()

# Подбор параметров ARIMA (p, d, q) с использованием функции auto_arima
# from pmdarima import auto_arima
# auto_model = auto_arima(df['total_load'], seasonal=False, trace=True)
# print(auto_model.summary())

# Обучение модели ARIMA
model = ARIMA(df['num_val'], order=(5, 1, 2))  # Пример: p=5, d=1, q=2
model_fit = model.fit()
print(model_fit.summary())



# Прогнозирование на 2 года (730 дней)



In [ ]:
forecast_steps = 365 * 2
forecast = model_fit.get_forecast(steps=forecast_steps)
forecast_index = pd.date_range(start=df.index[-1], periods=forecast_steps+1, inclusive='right')


# Прогнозированные значения и доверительные интервалы


In [ ]:
forecast_values = forecast.predicted_mean
confidence_intervals = forecast.conf_int()


# Визуализация


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df['num_val'], label='Исторические данные')
plt.plot(forecast_index, forecast_values, label='Прогноз')
plt.fill_between(forecast_index, confidence_intervals.iloc[:, 0], confidence_intervals.iloc[:, 1], color='pink', alpha=0.3)
plt.title('Прогноз загруженности станций на 2 года вперед')
plt.xlabel('Дата')
plt.ylabel('Общая загруженность')
plt.legend()
plt.show()